# Tests of zfturbo script "Santander Battle"

In [15]:
import datetime
import os
from collections import defaultdict
import operator

In [16]:
DATA_PATH = "data"

In [17]:
f = open(os.path.join(DATA_PATH, "train_ver2.csv"), 'r')
first_line = f.readline().strip()
# Remove " symbols in the header. Header is like : "a","b","c"...
first_line = first_line.replace("\"", "")
# map names <-> target labels
map_names = first_line.split(",")[24:]
print "MAP NAMES : ", map_names

MAP NAMES :  ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1', 'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']


In [18]:
# Normal variables
customer = dict()
best = defaultdict(lambda: defaultdict(int))
overallbest = defaultdict(int)

# Validation variables
customer_valid = dict()
best_valid = defaultdict(lambda: defaultdict(int))
overallbest_valid = defaultdict(int)

valid_part = []

In [23]:
# Calc counts
total = 0
while 1:
    line = f.readline()[:-1]
    total += 1

    if line == '':
        break

    # Parse line : 
    tmp1 = line.split("\"")
    arr = tmp1[0][:-1].split(",") + [tmp1[1]] + tmp1[2][1:].split(',')
    arr = [a.strip() for a in arr]

    # Normal part
    add_data_to_main_arrays(arr, best, overallbest, customer)

    
    # Valid part
    if arr[0] != '2016-05-28':
        add_data_to_main_arrays(arr, best_valid, overallbest_valid, customer_valid)
    else:
        valid_part.append(arr)

    if total % 1000000 == 0:
        print('Process {} lines ...'.format(total))
        break

Process 1000000 lines ...


In [ ]:
f.close()

In [25]:
print('Sort best arrays...')
print('Hashes num: ', len(best))
print('Hashes valid num: ', len(best_valid))
print('Valid part: ', len(valid_part))

Sort best arrays...
('Hashes num: ', 28639)
('Hashes valid num: ', 28639)
('Valid part: ', 0)


In [28]:
print overallbest
print overallbest.items()
print sorted(overallbest.items(), key=operator.itemgetter(1), reverse=True)

defaultdict(<type 'int'>, {0: 89, 1: 20, 2: 489653, 3: 313, 4: 59848, 5: 7766, 6: 8508, 7: 106732, 8: 36368, 9: 1625, 10: 1648, 11: 36589, 12: 61106, 13: 14227, 14: 5012, 15: 7448, 16: 2359, 17: 38414, 18: 38652, 19: 20471, 20: 3242, 21: 42838, 22: 47143, 23: 100539})
[(0, 89), (1, 20), (2, 489653), (3, 313), (4, 59848), (5, 7766), (6, 8508), (7, 106732), (8, 36368), (9, 1625), (10, 1648), (11, 36589), (12, 61106), (13, 14227), (14, 5012), (15, 7448), (16, 2359), (17, 38414), (18, 38652), (19, 20471), (20, 3242), (21, 42838), (22, 47143), (23, 100539)]
[(2, 489653), (7, 106732), (23, 100539), (12, 61106), (4, 59848), (22, 47143), (21, 42838), (18, 38652), (17, 38414), (11, 36589), (8, 36368), (19, 20471), (13, 14227), (6, 8508), (5, 7766), (15, 7448), (14, 5012), (20, 3242), (16, 2359), (10, 1648), (9, 1625), (3, 313), (0, 89), (1, 20)]


In [30]:
# Normal
best, overallbest = sort_main_arrays(best, overallbest)

# Valid
best_valid, overallbest_valid = sort_main_arrays(best_valid, overallbest_valid)

In [ ]:
map7 = 0.0
print('Validation...')
for arr1 in valid_part:
    predicted = get_predictions(arr1, best_valid, overallbest_valid, customer_valid)
    real = get_real_values(arr1, customer_valid)

    score = apk(real, predicted)
    map7 += score

if len(valid_part) > 0:
    map7 /= len(valid_part)
print('Predicted score: {}'.format(map7))

In [34]:
for a in best_valid[('ES', 'V', '86', '0', '01 - TOP', 'N', '1', 'S', 'SEGOVIA')]:
    print a

(12, 1)
(18, 1)
(11, 1)
(4, 1)
(22, 1)


In [ ]:
def get_predictions(arr1, best, overallbest, customer):

    predicted = []

    hash1 = get_hash(arr1, 0)
    hash2 = get_hash(arr1, 1)
    ncodpers = arr1[1]

    # hash 1
    if len(predicted) < 7:
        if hash1 in best:
            for a in best[hash1]:
                # If user is not new
                if ncodpers in customer:
                    if customer[ncodpers][a[0]] == '1':
                        continue
                if a[0] not in predicted:
                    predicted.append(a[0])
                    if len(predicted) == 7:
                        break

    # hash 2
    if len(predicted) < 7:
        if hash2 in best:
            for a in best[hash2]:
                # If user is not new
                if ncodpers in customer:
                    if customer[ncodpers][a[0]] == '1':
                        continue
                if a[0] not in predicted:
                    predicted.append(a[0])
                    if len(predicted) == 7:
                        break

    # overall
    if len(predicted) < 7:
        for a in overallbest:
            # If user is not new
            if ncodpers in customer:
                if customer[ncodpers][a[0]] == '1':
                    continue
            if a[0] not in predicted:
                predicted.append(a[0])
                if len(predicted) == 7:
                    break

    return predicted

In [ ]:
def get_real_values(arr1, customer):
    real = []
    ncodpers = arr1[1]
    arr2 = arr1[24:]

    for i in range(len(arr2)):
        if arr2[i] == '1':
            if ncodpers in customer:
                if customer[ncodpers][i] == '0':
                    real.append(i)
            else:
                real.append(i)
    return real

In [ ]:
def apk(actual, predicted, k=7):
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

In [6]:
def get_hash(arr, type = 0):
    (fecha_dato, ncodpers, ind_empleado,
    pais_residencia, sexo, age,
    fecha_alta, ind_nuevo, antiguedad,
    indrel, ult_fec_cli_1t, indrel_1mes,
    tiprel_1mes, indresi, indext,
    conyuemp, canal_entrada, indfall,
    tipodom, cod_prov, nomprov,
    ind_actividad_cliente, renta, segmento) = arr[:24]

    if type == 0:
        return (pais_residencia, sexo, age, ind_nuevo, segmento, ind_empleado, ind_actividad_cliente, indresi, nomprov)
    else:
        return (pais_residencia, sexo, age, ind_nuevo, segmento, ind_empleado, ind_actividad_cliente, indresi)

hash : client description (place of residence, gender, age, etc)

best : map between client description and stat of popular products

overallbest : stats of popular products

customer : latest command

In [19]:
def sort_main_arrays(best, overallbest):
    out = dict()
    for b in best:
        arr = best[b]
        srtd = sorted(arr.items(), key=operator.itemgetter(1), reverse=True)
        out[b] = srtd
    best = out
    overallbest = sorted(overallbest.items(), key=operator.itemgetter(1), reverse=True)
    return best, overallbest

In [20]:
def add_data_to_main_arrays(arr, best, overallbest, customer):
    ncodpers = arr[1]
    #print "CLIENT : ", ncodpers
    hash1 = get_hash(arr, 0)
    #print "HASH 1 : ", hash1
    hash2 = get_hash(arr, 1)
    #print "HASH 2 : ", hash2
    part = arr[24:]
    for i in range(24):
        if part[i] == '1':
            if ncodpers in customer:
                if customer[ncodpers][i] == '0':
                    best[hash1][i] += 1
                    best[hash2][i] += 1
                    overallbest[i] += 1
            else:
                best[hash1][i] += 1
                best[hash2][i] += 1
                overallbest[i] += 1
    customer[ncodpers] = part

In [21]:
def verbose():
    print "\nBEST : ", best.items()
    print "\nOVERALLBEST : ", overallbest.items()
    print "\nCUSTOMER : ",customer.items()

In [22]:
verbose()


BEST :  []

OVERALLBEST :  []

CUSTOMER :  []


In [35]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.DataFrame